# Cargamos variables de entorno

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

result = load_dotenv(find_dotenv(), override=True)

if result:
    print("Variables de entorno cargadas exitosamente.")
else:
    print("No se pudo cargar el archivo .env.")

    # Crear el archivo .env con las variables deseadas si no existe
    with open(".env", "w") as f:
        f.write(f"OPENAI_API_KEY=MY_OPENAI_API_KEY\n")
        f.write(f"PINECONE_API_KEY=MY_PINECONE_API_KEY\n")
        f.write(f"PINECONE_ENVIRONMENT=MY_PINECONE_ENVIRONMENT\n")

    print("Se creó el archivo .env con las variables iniciales.")
    # Vuelve a cargar las variables de entorno después de crear el archivo
    load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY") or "MY_OPENAI_API_KEY"
os.environ['PINECONE_API_KEY'] =  os.getenv("PINECONE_API_KEY") or "MY_PINECONE_API_KEY"
os.environ["PINECONE_ENVIRONMENT"] = os.getenv("PINECONE_ENVIRONMENT") or "MY_PINECONE_ENVIRONMENT"

# Transformacion de codificacion de .txt (Utf-8)

In [ ]:
import os
import ftfy

def fix_encoding(text):
    # Utilizar ftfy para corregir la codificación del texto
    fixed_text = ftfy.fix_text(text)
    return fixed_text

def convert_to_utf8(text, new_file_path):
    with open(new_file_path, 'w', encoding='utf-8') as f:
        f.write(text)

    print(f"El texto ha sido convertido a UTF-8 y guardado en '{new_file_path}'.")

def split_document(document):
    # Dividir el documento en párrafos usando el punto final como delimitador
    paragraphs = document.split(".")
    return paragraphs

def process_fragment(fragment):
    # Procesar cada párrafo
    # En este ejemplo, simplemente eliminaremos los caracteres especiales, convertiremos el texto a minúsculas
    # y eliminaremos los saltos de línea
    processed_fragment = fragment.lower()  # Convertir a minúsculas
    processed_fragment = ''.join(filter(str.isalnum, processed_fragment))  # Eliminar caracteres no alfanuméricos
    processed_fragment = processed_fragment.replace('\n', '')  # Eliminar saltos de línea
    return processed_fragment

def convert_directory_to_utf8(input_directory, output_directory):
    # Asegúrate de que el directorio de salida exista
    os.makedirs(output_directory, exist_ok=True)

    # Procesar cada archivo en el directorio de entrada
    for filename in os.listdir(input_directory):
        file_path = os.path.join(input_directory, filename)
        new_file_path = os.path.join(output_directory, filename)
        if os.path.isfile(file_path) and filename.endswith(".txt"):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    document = f.read()
            except UnicodeDecodeError:
                # Si la codificación utf-8 falla, intenta con latin-1
                with open(file_path, 'r', encoding='latin-1') as f:
                    document = f.read()

            paragraphs = split_document(document)
            processed_document = ''
            for paragraph in paragraphs:
                processed_paragraph = process_fragment(paragraph)
                processed_document += processed_paragraph

            convert_to_utf8(document, new_file_path)

# Directorios de entrada y salida
input_directory = "./../TXT_no_UTF8"
output_directory = "./../TXT_UTF8"

# Convertir todos los archivos de texto en el directorio de entrada a UTF-8
convert_directory_to_utf8(input_directory, output_directory)

print(f"Todos los archivos en '{input_directory}' han sido convertidos a UTF-8 y guardados en '{output_directory}'.")


# Creacion de Embeddings

In [ ]:
#Cargar librerias para cargar los documentos

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.llms.openai import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
def  documents_llm_model(output_directory):
    all_documents = []

    # Procesar cada archivo en el directorio
    for filename in os.listdir(output_directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(output_directory, filename)
            
            # Crear un TextLoader para cargar el archivo
            loader = TextLoader(file_path, autodetect_encoding=True)
            documents = loader.load()
            
            # Dividir los documentos en fragmentos de texto con un tamaño máximo de 800 tokens
            text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
            docs = text_splitter.split_documents(documents)
            
            # Agregar los documentos procesados a la lista general
            all_documents.extend(docs)

    embeddings = OpenAIEmbeddings()
    # Create vectors
    vectorstore = FAISS.from_documents(all_documents, embeddings)  # Usar all_documents en lugar de docs
    # Persist the vectors locally on disk
    vectorstore.save_local("faiss_index")

documents_llm_model(output_directory)